In [1]:
import pandas as pd
import subprocess
import glob
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn import metrics

# Part 1: Run best train-set only model on test set to ensure generalizability

In [4]:
# can do with subprocess, in the terminal, or however you want
# test just train set model on test set

# activate_command = 'conda activate chemprop; '
# for i in [16]: #Just do 16 - this was the best model by both kill and average AUC
#     print(i)
#     run_command = 'python predict.py --test_path ../nontoxic_stat_phase_killing_abx/out/data_prep_for_ml_testset.csv --checkpoint_dir ../nontoxic_stat_phase_killing_abx/models/hyperopt_gnn_models/' + str(i) + ' --preds_path ../nontoxic_stat_phase_killing_abx/models/hyperopt_gnn_models/valtestonpreds/' + str(i) + '.csv --features_path ../nontoxic_stat_phase_killing_abx/out/data_prep_for_ml_testset.npz --no_features_scaling --smiles_column SMILES --gpu 0 --ensemble_variance'
#     full_command = activate_command + run_command
#     test = subprocess.run(full_command, cwd="../chemprop/", shell=True, capture_output=True)
#     print(test)

16
CompletedProcess(args='conda activate chemprop; python predict.py --test_path ../ian/round3finalval08162021_TEST.csv --checkpoint_dir ../ian/round3finalval08162021_models/16 --preds_path ../ian/round3finalval08162021_models/valtestonpreds/16.csv --features_path ../ian/round3finalval08162021_TESTFTS.npz --no_features_scaling --smiles_column SMILES --gpu 0 --ensemble_variance', returncode=0, stdout=b'Loading training args\nLoading data\nValidating SMILES\nTest size = 1,333\nPredicting with an ensemble of 3 models\nLoading pretrained parameter "encoder.encoder.0.cached_zero_vector".\nLoading pretrained parameter "encoder.encoder.0.W_i.weight".\nLoading pretrained parameter "encoder.encoder.0.W_h.weight".\nLoading pretrained parameter "encoder.encoder.0.W_o.weight".\nLoading pretrained parameter "encoder.encoder.0.W_o.bias".\nLoading pretrained parameter "ffn.1.weight".\nLoading pretrained parameter "ffn.1.bias".\nLoading pretrained parameter "ffn.4.weight".\nLoading pretrained paramete

In [3]:
# grab the parameters of the best model
cols = ['num', 'init_lr', 'dropout', 'hiddensize', 'ffn num layers', 'depth']
df = pd.DataFrame(columns = cols)

# parameters are hardcoded, but we'll keep the same format as before
initlr = [1e-3]
dropout = [0.1]
hiddensize = [1000]
ffnnumlayers = [2]
depth = [4]

# again, "loop" through model parameters (even though just one)
model_dir = 0
for l in initlr:
    for d in dropout:
        for h in hiddensize:
            for f in ffnnumlayers:
                for de in depth:
                    new_row = [model_dir, l, d, h, f, de]
                    new = pd.DataFrame(new_row).T
                    new.columns = cols
                    df = pd.concat([df, new], ignore_index = True)   
                    model_dir = model_dir + 1
df

,num,init_lr,dropout,hiddensize,ffn num layers,depth
0,0.0,0.001,0.1,1000.0,2.0,4.0


# Part 2: Evaluate predictions on 20% test set

In [4]:
# streamlined function to not plot, but just return auroc and pr metrics
def modeleval(y_true, y_pred):
    
    new_ytrue = []
    new_ypred = []
    for x, y in zip(y_true, y_pred):
        if y != 'Invalid SMILES':
            new_ytrue.append(x)
            new_ypred.append(y)
    y_true = new_ytrue
    y_pred = new_ypred
    
    auroc = float(roc_auc_score(y_true, y_pred))
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)
    pr = float(auc(recall,precision))    
    
    return(auroc, pr)

In [15]:
# this code is amenable to running many models on the same test set
# here, just 1 model, but still keep same table output format
true = pd.read_csv('../out/data_prep_for_ml_testset.csv')
display(true)
print('total test compounds: ', len(true))

# get total number of inhibition and killing compounds in ground truth test set
true_kill = list(true['hit_kill'])
true_inh = list(true['hit_inh'])
print('total inh compounds: ', sum(true_inh))
print('total kill compounds: ', sum(true_kill))

# keep track of metrics (again, not necessary to do in list format; keeping in same format for consistency)
killrocs = []
inhrocs = []
avgrocs = []
killprs = []
inhprs = []
avgprs = []

# loop through models and grab metrics
for i in [16]:
    test = pd.read_csv('../models/hyperopt_gnn_models/valtestonpreds/' + str(i) + '.csv')
    display(test)
    print(len(test))

    # grab y-pred for killing and inhibition
    test_kill = [float(x) if x != 'Invalid SMILES' else x for x in list(test['hit_kill'])]
    test_inh = [float(x) if x != 'Invalid SMILES' else x for x in list(test['hit_inh'])]

    # calculate ROC and PR for killing and inhibition
    roc_kill, pr_kill = modeleval(true_kill, test_kill)
    roc_inh, pr_inh = modeleval(true_inh, test_inh)

    # calculate average ROC and PR over killing and inhibition contexts
    avgroc = np.average([roc_kill, roc_inh])
    avgpr = np.average([pr_kill, pr_inh])

    # keep track of metrics
    killrocs.append(roc_kill)
    inhrocs.append(roc_inh)
    avgrocs.append(avgroc)
    killprs.append(pr_kill)
    inhprs.append(pr_inh)
    avgprs.append(avgpr)

# make dataframe for results to live
df['inh auroc'] = inhrocs
df['kill auroc'] = killrocs
df['avg auroc'] = avgrocs
df['inh pr'] = inhprs
df['kill pr'] = killprs
df['avg pr'] = avgprs

,Name,SMILES,hit_inh,hit_kill
0,triclosan,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,1.0,1.0
1,amiselimod,CCCCCCCOc1ccc(CCC(N)(CO)CO)cc1C(F)(F)F,1.0,1.0
2,silver-sulfadiazine,Nc1ccc(cc1)S(=O)(=O)N([Ag])c1ncccn1,1.0,1.0
3,visomitin,CC1=C(C)C(=O)C(CCCCCCCCCC[P+](c2ccccc2)(c2cccc...,1.0,1.0
4,colistin,CC[C@@H](C)CCCC(=O)N[C@@H](CCN)C(=O)N[C@@H]([C...,1.0,1.0
...,...,...,...,...
1334,tetrandrine,COc1ccc2C[C@@H]3N(C)CCc4cc(OC)c(OC)c(Oc5cc6[C@...,0.0,0.0
1335,tozasertib,CN1CCN(CC1)c1cc(Nc2cc(C)n[nH]2)nc(Sc2ccc(NC(=O...,0.0,0.0
1336,dexniguldipine,COC(=O)C1=C(C)NC(C)=C([C@@H]1c1cccc(c1)[N+]([O...,0.0,0.0
1337,CP-93129,O=c1ccc2[nH]cc(C3=CCNCC3)c2[nH]1 |t:8|,0.0,0.0


total test compounds:  1339
total inh compounds:  32.0
total kill compounds:  11.0


,Name,SMILES,hit_inh,hit_kill,hit_inh_epi_unc,hit_kill_epi_unc
0,triclosan,Oc1cc(Cl)ccc1Oc1ccc(Cl)cc1Cl,0.010232437304997196,0.015759015159953076,4.8423454846631713e-05,0.0001417453234915892
1,amiselimod,CCCCCCCOc1ccc(CCC(N)(CO)CO)cc1C(F)(F)F,0.023644243832677603,0.1439292567471663,0.000503677778492586,0.003901027148760731
2,silver-sulfadiazine,Nc1ccc(cc1)S(=O)(=O)N([Ag])c1ncccn1,0.007268386462480218,0.01222871613274871,6.833031345301617e-05,0.00021030408804067354
3,visomitin,CC1=C(C)C(=O)C(CCCCCCCCCC[P+](c2ccccc2)(c2cccc...,0.10377278675635655,0.2440233255426089,0.0028742694487902984,0.01585495220781442
4,colistin,CC[C@@H](C)CCCC(=O)N[C@@H](CCN)C(=O)N[C@@H]([C...,0.22679844498634338,0.3161508689324061,0.03631240506991462,0.03202050160837105
...,...,...,...,...,...,...
1334,tetrandrine,COc1ccc2C[C@@H]3N(C)CCc4cc(OC)c(OC)c(Oc5cc6[C@...,0.025173816209038097,0.09805228328332305,5.005499911874066e-05,0.004785999939124593
1335,tozasertib,CN1CCN(CC1)c1cc(Nc2cc(C)n[nH]2)nc(Sc2ccc(NC(=O...,0.024109865708548266,0.041931728206388165,0.0007439247350902915,0.0013701027892702037
1336,dexniguldipine,COC(=O)C1=C(C)NC(C)=C([C@@H]1c1cccc(c1)[N+]([O...,0.0005440219610287992,0.0024397528727414888,3.685802544669385e-07,9.886969220352523e-06
1337,CP-93129,O=c1ccc2[nH]cc(C3=CCNCC3)c2[nH]1 |t:8|,0.010808713650779586,0.013551304428650232,6.0857951310980854e-05,0.0001359409407415994


1339


In [16]:
# display results df
df

,num,init_lr,dropout,hiddensize,ffn num layers,depth,inh auroc,kill auroc,avg auroc,inh pr,kill pr,avg pr
0,0.0,0.001,0.1,1000.0,2.0,4.0,0.849947,0.739582,0.794765,0.504886,0.055449,0.280167
